# Null and Missing Values Notebook

This notebook will run you through the problem of missing and null values when doing data cleaning. There are 2 approaches to handling missing/null values. The first is deletion where you delete the entire column associated with the null value and the second involves imputation to fill the missing values. There are many kinds of imputation such as summary statistic, linear regression and K-Nearest Neighbours. For this notebook, we are going to look at summary statistic as KNN is yet to be taught. The first method involving deletion of the column runs into obvious issues of affecting the data model significantly. We will still explore both of the methods. It is helpful to drop data that have too many missing values as it is hard to gauge the correct values since there might not be enough information for imputation.

## Importing Libraries and Dataset
First, we import the necessary libraries and dataset. The dataset we are using for this assignment is Kaggle's spam Melbounre Housing dataset.

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

raw_data = pd.read_csv('Melbourne_housing_FULL.csv')
full_data = raw_data.select_dtypes(exclude=['object'])
full_data.head()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,NaN,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,-37.8014,144.9958,4019.0
1,2,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
2,2,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
3,3,NaN,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,NaN,-37.8114,145.0116,4019.0
4,3,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0


The cell below is a helper function that shows the number of data points that is missing by columns.

In [2]:
def missing_vals(data):
    missing_val_count_by_column = (data.isnull().sum())
    return (missing_val_count_by_column[missing_val_count_by_column > 0])
missing_vals(full_data)

Price             7610
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
Lattitude         7976
Longtitude        7976
Propertycount        3
dtype: int64

In [3]:
full_data.count()

Rooms            34857
Price            27247
Distance         34856
Postcode         34856
Bedroom2         26640
Bathroom         26631
Car              26129
Landsize         23047
BuildingArea     13742
YearBuilt        15551
Lattitude        26881
Longtitude       26881
Propertycount    34854
dtype: int64

## Dropping Columns and Rows

The count in the cell above shows that there are many columns with missing values present in the data. This part of the notebook approaches the problem by deletion of rows and columns.

Question : __Drop the variables of YearBuilt and Building Area for both the train and test data__

In [4]:
# Begin Solution
data = full_data.drop(columns = ['YearBuilt', 'BuildingArea'])
# End Solution
data.head(5)

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,Lattitude,Longtitude,Propertycount
0,2,NaN,2.5,3067.0,2.0,1.0,1.0,126.0,-37.8014,144.9958,4019.0
1,2,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,-37.7996,144.9984,4019.0
2,2,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,-37.8079,144.9934,4019.0
3,3,NaN,2.5,3067.0,3.0,2.0,1.0,0.0,-37.8114,145.0116,4019.0
4,3,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,-37.8093,144.9944,4019.0


In [5]:
missing_vals(data)

Price             7610
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
Lattitude         7976
Longtitude        7976
Propertycount        3
dtype: int64

Now we shall explore deletion and imputation with our existing dataset. Let's try to drop all the rows with a null value and then try to drop all the columns with null values. First, we shall explore dropping the rows.

In [6]:
dropped_rows = data.copy(deep=True)
# Begin Solution
dropped_rows.dropna(inplace=True)
# End Solution
print(missing_vals(dropped_rows))
assert missing_vals(dropped_rows).sum() == 0
dropped_rows.head()

Series([], dtype: int64)


,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,Lattitude,Longtitude,Propertycount
1,2,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,-37.7996,144.9984,4019.0
2,2,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,-37.8079,144.9934,4019.0
4,3,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,-37.8093,144.9944,4019.0
5,3,850000.0,2.5,3067.0,3.0,2.0,1.0,94.0,-37.7969,144.9969,4019.0
6,4,1600000.0,2.5,3067.0,3.0,1.0,2.0,120.0,-37.8072,144.9941,4019.0


There should be no missing values in the new dataset but now let us take a look at the number of points remaining in the dataset.

In [7]:
dropped_rows.count()

Rooms            17679
Price            17679
Distance         17679
Postcode         17679
Bedroom2         17679
Bathroom         17679
Car              17679
Landsize         17679
Lattitude        17679
Longtitude       17679
Propertycount    17679
dtype: int64

We notice that we have removed around half of the original dataset by dropping the rows with missing values. Consider the case where we did not drop the variables before dropping the rows with missing values. How many points would you expect?. Now lets us try this with columns. <br>
__What do you expect to see if we drop the columns with missing datapoints?__

In [8]:
dropped_cols = data.copy(deep=True)
# Begin Solution
dropped_cols.dropna(inplace=True, axis = 1)
# End Solution
print(missing_vals(dropped_cols))
assert missing_vals(dropped_cols).sum() == 0
dropped_cols.head()

Series([], dtype: int64)


,Rooms
0,2
1,2
2,2
3,3
4,3


For obvious reasons we cannot exactly run a model to find price based on the information with columns due to price having null values and therefore being dropped which makes the setup harder to implement when dropping columns in addition to the existence of only one column. It is also hard to compare the models from the dropped rows due to different dataset size which leads to a lower mean approximate error.

## Imputation of values

Now we are going to try imputation of values instead of deleting them. Refer to Simple Imputer from sklearn.impute for assistance involving this. Provided is the train-test split and the function to evaluate the dataset.

In [9]:
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

full_data = full_data.dropna(axis=0, subset=['Price'])
X_train, X_test, y_train, y_test = train_test_split(full_data.drop(columns = ['Price']), full_data['Price'], train_size=0.7, 
                                                    test_size=0.3, random_state=0)

In [10]:
# Begin Solution
my_imputer = SimpleImputer()
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_X_test = my_imputer.transform(X_test)
# End Solution
score_dataset(imputed_X_train, imputed_X_test, y_train, y_test)

183654.7563045506

Next we move on to different types of imputation. We are going to try KNN imputation here. Using the same train-test split implement KNN imputation using 4 nearest neighbours and uniform weights. 

In [11]:
# Begin Solution
KNN_imputer = KNNImputer(n_neighbors=4, weights="uniform")
KNN_X_train = KNN_imputer.fit_transform(X_train)
KNN_X_test = KNN_imputer.transform(X_test)
# End Solution
score_dataset(KNN_X_train, KNN_X_test, y_train, y_test)

184193.49712114123

Further avenues of exploring imputation include marking imputation as a feature and adding it to the regression in addition to different imputation models.